In [34]:
import os
import torch
os.environ["CUDA_VISIBLE_DEVICES"]="0"
print(torch.cuda.is_available())

True


In [35]:
import subprocess
import os
import pandas as pd
import json
import nltk
from tqdm import tqdm
import re
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)
import matplotlib.pyplot as plt

DEBUG= False

In [36]:
DATA_PATH = "/root/workspace/FAISS-Imputation/GITHUB/SolvER/data/DBLP-ACM/"
DATA_PATH_FOR_MATCHER="/DBLP-ACM"
CONFIG_PATH = "/root/workspace/FAISS-Imputation/GITHUB/SolvER/configs.json"

# Training Model

In [37]:
def train_again_from_scratch():
  os.system("CUDA_VISIBLE_DEVICES=0 python train_ditto.py --task " +str(DATA_PATH_FOR_MATCHER)+" --batch_size 32 --max_len 64 --lr 3e-5 --n_epochs 15 --finetuning  --lm distilbert  --fp16  --save_model  --da drop_col")

# Run the matche


In [38]:
def get_f1_values(s):
    load_f1_pattern = re.compile(r'load_f1\s*=\s*([0-9.]+)')
    real_f1_pattern = re.compile(r'real_f1\s*=\s*([0-9.]+)')
    tn_pattern = re.compile(r'TN\s*=\s*([0-9.]+)')
    tp_pattern = re.compile(r'TP\s*=\s*([0-9.]+)')
    fn_pattern = re.compile(r'FN\s*=\s*([0-9.]+)')
    fp_pattern = re.compile(r'FP\s*=\s*([0-9.]+)')
    fnr_pattern = re.compile(r'FNR\s*=\s*([0-9.]+)')
    rec_pattern = re.compile(r'REC\s*=\s*([0-9.]+)')

    load_f1_match = load_f1_pattern.search(s)
    real_f1_match = real_f1_pattern.search(s)
    tn_match = tn_pattern.search(s)
    tp_match = tp_pattern.search(s)
    fn_match = fn_pattern.search(s)
    fp_match = fp_pattern.search(s)
    fnr_match = fnr_pattern.search(s)
    rec_match = rec_pattern.search(s)
    if load_f1_match:
        load_f1_value = float(load_f1_match.group(1))
    else:
        load_f1_value = -1
    if real_f1_match:
        real_f1_value = float(real_f1_match.group(1))
    else:
        real_f1_value = -1
    if tn_match:
        tn_value = float(tn_match.group(1))
    else:
        tn_value = -1
    if tp_match:
        tp_value = float(tp_match.group(1))
    else:
        tp_value = -1
    if fn_match:
        fn_value = float(fn_match.group(1))
    else:
        fn_value = -1
    if fp_match:
        fp_value = float(fp_match.group(1))
    else:
        fp_value = -1
    if fnr_match:
        fnr_value = float(fnr_match.group(1))
    else:
        fnr_value = -1
    if rec_match:
        rec_value = float(rec_match.group(1))
    else:
        rec_value = -1
    return load_f1_value, real_f1_value, tn_value, tp_value, fn_value, fp_value, fnr_value, rec_value

In [39]:
def call_matcher(file, out_file):
  command = ['python', 'matcher.py', '--task', DATA_PATH_FOR_MATCHER, '--input_path', file, '--output_path',
             f'output/{out_file}.jsonl', '--lm', 'distilbert', '--max_len', '64', '--fp16', '--use_gpu', '--checkpoint_path', 'checkpoints/']
  result = subprocess.run(command, capture_output=True, text=True)
  if DEBUG:
    print(result.stdout)
    print(result.stderr)
  load_f1, real_f1, tn, tp, fn, fp, fnr, rec = get_f1_values(result.stdout)
  file_last=file.split('/')[-1]
  isImputed=1
  if "nonimputed" in file_last:
    isImputed=0
    file_last = file_last.replace("_nonimputed_final","")
  else:
    file_last = file_last.replace("_imputed_final","")
  file=file[:file.rfind('/')+1]+file_last
  seed=file_last.split("_")[3][:-4]
  subset = file_last.split("_")[2]
  return (file, subset, seed, isImputed, real_f1, tn, tp, fn, fp, fnr, rec)

In [40]:
def update_config(config_path, data_path, file):
    with open(config_path, 'r') as cfg:
        config = json.load(cfg)
        config[0]['name'] = DATA_PATH_FOR_MATCHER            
        config[0]['testset'] = data_path + file
        config[0]['trainset'] = "/root/workspace/FAISS-Imputation/GITHUB/SolvER/data/DBLP-ACM/train_DBLPACMS.txt"
        config[0]['validset'] = "/root/workspace/FAISS-Imputation/GITHUB/SolvER/data/DBLP-ACM/valid_DBLPACMS.txt"
    with open(config_path, 'w') as cfg_out:
        json.dump(config, cfg_out, indent=4)

In [ ]:
# Change on-the-fly the configs.json file
all_files = os.listdir(DATA_PATH)
txt_files = [f for f in all_files if f.endswith(".txt")]
results = []
seeds= 98, 409, 1753, 8424, 2782, 6004, 3581, 5553, 4721, 7179
all_percentages=["0.0", "0.1", "0.2", "0.3", "0.4"]
percentages=all_percentages
c=len(txt_files)
update_config(CONFIG_PATH, DATA_PATH, "testing_file_0.0_98_imputed_final.txt")
train_again_from_scratch()
for s in seeds:
    for p in percentages:
        for file in tqdm(txt_files):
            if (str(s) in file) and ((str(p)+'_') in file) and ("testing_" in file): 
                print("Updating config for ", file)
                update_config(CONFIG_PATH, DATA_PATH, file)
                name_out = "output_"+file
                print("Processing ", file)
                results.append(call_matcher(file=DATA_PATH+file, out_file=name_out))
                c=c-1
                print("REMAINING FILES="+str(c))
df = pd.DataFrame(results, columns=['file', 'subset', 'seed', 'isImputed', 'real_f1', 'tn','tp', 'fn', 'fp', 'fnr', 'rec'])
print(df.head())
df.to_csv(f"{DATA_PATH}/Results_DBLPACM.csv", index=False)